## 定义模型3 Googlenet

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd

%matplotlib qt

In [2]:
import torch
model_accuracies = {
    'googlenet': []
}
torch.cuda.empty_cache()
# 定义批量大小
batch_size = 60

# 数据增强和预处理
transform3 = transforms.Compose([
    transforms.Resize((224, 224)),##图像大小转化，转化为各深度学习分类器要求的图像大小
    transforms.ToTensor(),## 将图像转换为PyTorch的张量格式，并将像素值从0-255缩放到0-1之间
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])## 标准化图像，使每个通道的像素值分布在[-1, 1]之间。
])

In [3]:

# 加载数据集
data_dir = r'D:\深度学习练习\flower_photos\flower_photos'
train_dataset = datasets.ImageFolder(root=data_dir, transform=transform3)
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [4]:
# 定义Inception模块
class Inception(nn.Module):#一个复合模块，其作用和卷积层和池化层相当
    def __init__(self, in_channels, ch1x1, ch3x3red, ch3x3, ch5x5red, ch5x5, pool_proj):
        super(Inception, self).__init__()
        self.branch1 = nn.Conv2d(in_channels, ch1x1, kernel_size=1)
        
        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels, ch3x3red, kernel_size=1),
            nn.Conv2d(ch3x3red, ch3x3, kernel_size=3, padding=1)
        )
        
        self.branch3 = nn.Sequential(
            nn.Conv2d(in_channels, ch5x5red, kernel_size=1),
            nn.Conv2d(ch5x5red, ch5x5, kernel_size=5, padding=2)
        )
        
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, pool_proj, kernel_size=1)
        )

    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)
        outputs = [branch1, branch2, branch3, branch4]
        return torch.cat(outputs, 1)

In [5]:
# 定义GoogLeNet模型
class GoogLeNet(nn.Module):
    def __init__(self, num_classes=5):
        super(GoogLeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=1)
        self.conv3 = nn.Conv2d(64, 192, kernel_size=3, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.inception3a = Inception(192, 64, 96, 128, 16, 32, 32)#参数介绍见下
        self.inception3b = Inception(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.inception4a = Inception(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = Inception(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = Inception(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = Inception(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = Inception(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        self.inception5a = Inception(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = Inception(832, 384, 192, 384, 48, 128, 128)
        
        self.avgpool = nn.AvgPool2d(kernel_size=8, stride=1)
        self.dropout = nn.Dropout(p=0.4)
        self.fc = nn.Linear(1024, num_classes)
# self.inception3a = Inception(192, 64, 96, 128, 16, 32, 32)
#192：输入特征图的通道数。这表示Inception模块接收的输入特征图具有192个通道。
#64：1x1卷积层的输出通道数。这表示在Inception模块的第一个分支中，将通过1x1卷积层减少输入特征图的通道数，输出特征图将有64个通道。
#96：3x3卷积层之前的1x1卷积层的输出通道数。在Inception模块的第二个分支中，首先通过一个1x1卷积层减少通道数到96个，然后再通过一个3x3卷积层。
#128：3x3卷积层的输出通道数。在第二个分支中，3x3卷积层将输出128个通道的特征图。
#16：5x5卷积层之前的1x1卷积层的输出通道数。在Inception模块的第三个分支中，先通过一个1x1卷积层减少通道数到16个，然后再通过一个5x5卷积层。
#32：5x5卷积层的输出通道数。在第三个分支中，5x5卷积层将输出32个通道的特征图。
#32：最大池化层后接1x1卷积层的输出通道数。在Inception模块的第四个分支中，先通过最大池化层进行下采样，然后通过一个1x1卷积层增加通道数到32个。   
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.maxpool1(x)
        #print("After maxpool1:", x.size())
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = self.maxpool2(x)
        #print("After maxpool2:", x.size())
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)
        #print("After maxpool3:", x.size())
        x = self.inception4a(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        x = self.inception4e(x)
        x = self.maxpool4(x)
        #print("After maxpool4:", x.size())
        x = self.inception5a(x)
        x = self.inception5b(x)
        #print("After maxpool5:", x.size())
        x = self.avgpool(x)
        #print("After avgpool:", x.size())
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc(x)
        
        return x

参数检查
from torchsummary import summary
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
##### 使用summary函数查看模型的详细信息，这里假设输入尺寸为(3, 224, 224)
model = GoogLeNet(num_classes=5).to(device)
summary(model, (3, 224, 224))

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 初始化模型、损失函数和优化器
model = GoogLeNet(num_classes=5).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# 训练模型
epochs = 40
train_losses = []
val_losses = []
accuracy_per_epoch = []  # 存储每个epoch的准确率

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_losses.append(running_loss / len(train_loader))
    print(f"Epoch {epoch+1}, Train Loss: {running_loss / len(train_loader)}")

    # 验证模型
    model.eval()
    val_running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    val_losses.append(val_running_loss / len(val_loader))
    accuracy = 100 * correct / total
    accuracy_per_epoch.append(accuracy)
    print(f"Epoch {epoch+1}, Val Loss: {val_running_loss / len(val_loader)}")
    print(f"Epoch {epoch+1}, Validation Accuracy: {accuracy:.2f}%")

# 更新字典中的准确率数据
model_accuracies['googlenet'] = accuracy_per_epoch

Using device: cuda
Epoch 1, Train Loss: 1.468608140297558
Epoch 1, Val Loss: 1.2536434971767922
Epoch 1, Validation Accuracy: 44.69%
Epoch 2, Train Loss: 1.1769372263680333
Epoch 2, Val Loss: 1.1258400419484014
Epoch 2, Validation Accuracy: 48.50%
Epoch 3, Train Loss: 1.1014044563407484
Epoch 3, Val Loss: 1.0462428098139556
Epoch 3, Validation Accuracy: 53.54%
Epoch 4, Train Loss: 1.0593450276747993
Epoch 4, Val Loss: 1.2425918319950933
Epoch 4, Validation Accuracy: 48.23%
Epoch 5, Train Loss: 1.0430262393277625
Epoch 5, Val Loss: 0.9203289850898411
Epoch 5, Validation Accuracy: 61.04%
Epoch 6, Train Loss: 1.015951033519662
Epoch 6, Val Loss: 0.9786207442698271
Epoch 6, Validation Accuracy: 59.13%
Epoch 7, Train Loss: 0.9868745499331019
Epoch 7, Val Loss: 0.9197262473728346
Epoch 7, Validation Accuracy: 59.95%
Epoch 8, Train Loss: 0.9603148815424546
Epoch 8, Val Loss: 0.983420835888904
Epoch 8, Validation Accuracy: 57.22%
Epoch 9, Train Loss: 0.9461500916791998
Epoch 9, Val Loss: 0.887

In [7]:

# 保存模型权重
torch.save(model.state_dict(), 'googlenet_weights.pth')
# 保存模型结构和权重
torch.save(model, 'googlenet_model.pth')
accuracies_list = model_accuracies['googlenet']

# 创建一个包含epoch编号和对应准确率的字典
data = {'Epoch': list(range(1, len(accuracies_list) + 1)),
        'Accuracy': accuracies_list}

# 将字典转换为DataFrame
df = pd.DataFrame(data)

# 现在你可以将DataFrame保存为Excel文件
df.to_excel('googlenet_accuracies.xlsx', index=False)
# 清理内存
del model
del inputs
del labels
torch.cuda.empty_cache()

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# 实例化模型
model = GoogLeNet(num_classes=5).to(device)

# 加载模型权重
model.load_state_dict(torch.load('googlenet_weights.pth'))

# 设置为评估模式
model.eval()

Using device: cuda


GoogLeNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
  (conv3): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (inception3a): Inception(
    (branch1): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
    (branch2): Sequential(
      (0): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(96, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (branch3): Sequential(
      (0): Conv2d(192, 16, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    )
    (branch4): Sequential(
      (0): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
      (1): Conv2d(192, 32, kernel_size=(1, 1

In [10]:
import torch
import csv
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

# 确保模型处于评估模式
model.eval()

# 初始化用于存储预测和真实标签的列表
all_preds = []
all_labels = []

# 不计算梯度进行预测
with torch.no_grad():
    for inputs, labels in tqdm(val_loader, desc="Evaluating"):
        # 将数据移动到GPU，如果可用
        inputs, labels = inputs.to(device), labels.to(device)

        # 前向传播
        outputs = model(inputs)

        # 获取预测概率最高的类别
        _, predicted = torch.max(outputs.data, 1)

        # 将预测结果和真实标签添加到列表中
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 计算准确率、精确率、召回率和F1分数
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted', zero_division=1)
recall = recall_score(all_labels, all_preds, average='weighted', zero_division=1)
f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=1)

# 将结果保存到CSV文件
results = {
    'Accuracy': accuracy,
    'Precision': precision,
    'Recall': recall,
    'F1 Score': f1
}

# 写入CSV文件
with open('GoogLeNet评估.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Metric', 'Value'])
    for metric, value in results.items():
        writer.writerow([metric, value])

print("Results have been saved to GoogLeNet评估.csv")

Evaluating: 100%|██████████| 23/23 [00:02<00:00,  9.38it/s]

Results have been saved to GoogLeNet评估.csv
